In [1]:
import sympy as sym
from scipy.constants import g
import numpy as np

In [31]:
# Maximum battery energy
E_max_J = sym.symbols("E_max") # 7.2e+8 J

# Electrical power required
eta_, rho_cruise_kgpm3, v_cruise_M3_mps, S_wing_m2, cd_nonwing_, cd_wing_, CL_, e_, AR_ = \
sym.symbols("\eta_{prop} rho_cruise v^{M3}_{cruise} S_wing cd_nonwing cd_wing C_L^{cruise} e AR")

# Wing geometry
b_m, AR_ = sym.symbols("b AR")
S_wing_m2 = (b_m**2) / AR_

P_elec_W = (1/eta_) * (0.5 * rho_cruise_kgpm3 * (v_cruise_M3_mps**3)) * S_wing_m2 * \
            (cd_nonwing_ + cd_wing_ + ((CL_ ** 2)/(np.pi * e_ * AR_)))

P_elec_W

0.5*b**2*rho_cruise*v^{M3}_{cruise}**3*(cd_nonwing + cd_wing + 0.318309886183791*C_L^{cruise}**2/(AR*e))/(AR*\eta_{prop})

In [32]:
Endurance = E_max_J / P_elec_W
Endurance

2.0*AR*E_max*\eta_{prop}/(b**2*rho_cruise*v^{M3}_{cruise}**3*(cd_nonwing + cd_wing + 0.318309886183791*C_L^{cruise}**2/(AR*e)))

In [33]:
# lap time
t_lap_M3_s = sym.symbols("t^{M3}_{lap}")

theta_rad, psi_dot_M3_radps = sym.symbols("theta \dot{\psi}_{M3}")
d_m = sym.symbols("d")

A_n_ = sym.symbols("A_n")
psi_dot_M3_radps = g * sym.sqrt((A_n_ ** 2) - 1) / v_cruise_M3_mps

t_lap_M3_s = (theta_rad / psi_dot_M3_radps) + (d_m / v_cruise_M3_mps)

t_lap_M3_s

d/v^{M3}_{cruise} + 0.101971621297793*theta*v^{M3}_{cruise}/sqrt(A_n**2 - 1)

In [34]:
n_laps = Endurance / t_lap_M3_s
n_laps

2.0*AR*E_max*\eta_{prop}/(b**2*rho_cruise*v^{M3}_{cruise}**3*(d/v^{M3}_{cruise} + 0.101971621297793*theta*v^{M3}_{cruise}/sqrt(A_n**2 - 1))*(cd_nonwing + cd_wing + 0.318309886183791*C_L^{cruise}**2/(AR*e)))

In [35]:
n_laps.free_symbols

{AR,
 A_n,
 C_L^{cruise},
 E_max,
 \eta_{prop},
 b,
 cd_nonwing,
 cd_wing,
 d,
 e,
 rho_cruise,
 theta,
 v^{M3}_{cruise}}

In [51]:
all_params = {
    # Parameters
    "AR": np.arange(3, 5 + 0.5, 0.5),
    "A_n": np.arange(2.5, 5 + 0.5, 0.5),
    "C_L^{cruise}": np.arange(0.5, 0.8 + 0.1, 0.1),
    "b": np.arange(1, 1.5 + 0.5, 0.5),
    "v^{M3}_{cruise}": np.arange(15, 25 + 2),
    "cd_nonwing": np.arange(0.04, 0.07, 0.01),
    "cd_wing": np.arange(0.04, 0.07, 0.01),
    "e": np.arange(0.7, 0.9, 0.1),
    
    # Constants
    "rho_cruise": [1.225],
    "theta": [4 * np.pi],
    "d": [610],
    "E_max": [7.2e+8],
    "\eta_{prop}": [0.4]
}

In [52]:
from subs_generator import generate, calculate
all_subs = generate(all_params)

In [59]:
len(all_subs)

138240

In [60]:
all_calc = calculate(n_laps, all_subs)

138239/138240	100.00%
Sorting...


In [62]:
all_calc[0] 

{'AR': 5.0,
 'A_n': 5.0,
 'C_L^{cruise}': 0.5,
 'b': 1.0,
 'v^{M3}_{cruise}': 15,
 'cd_nonwing': 0.04,
 'cd_wing': 0.04,
 'e': 0.8999999999999999,
 'rho_cruise': 1.225,
 'theta': 12.566370614359172,
 'd': 610,
 'E_max': 720000000.0,
 '\\eta_{prop}': 0.4,
 'total': 159926.528611540}

In [63]:
Endurance.subs(all_calc[0])

7131152.25520232

In [64]:
P_elec_W.subs(all_calc[0])

100.965450495710